In [1]:
!pip3 install opencv-python 

# Imports

In [38]:
import matplotlib.pyplot as plt
import random
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import utils, optimizers
from PIL import Image
import shutil
from tensorflow.keras import models, layers
from tensorflow.keras.applications import efficientnet
from tensorflow.keras.callbacks import EarlyStopping

# Image Data Generator

In [2]:
root_dir = '../raw_data'
poses_list = os.listdir(root_dir) #Lists all folders in the root directory (raw_data folder)
poses_list

['Garland_Pose',
 'Peacock_Pose',
 'Training',
 'Four-Limbed_Staff',
 'Sitting_pose_1_(normal)',
 'Testing',
 'Staff_Pose_',
 'Low_Lunge_pose',
 'Standing_big_toe_hold']

In [3]:
# Creates Testing and Training directories
def create_train_val_dirs(root_path):
    for pose in poses_list:
        if pose == "Testing" or pose == "Training":
            pass
        else:
            os.makedirs(os.path.join(root_path, 'Training', pose))
            os.makedirs(os.path.join(root_path, 'Testing', pose))
            
            
#Tries to create new directories
#Errors if Training/Testing directories already exist - delete them before running this notebook!
try:
    create_train_val_dirs(root_path=root_dir)
except FileExistsError:
    print("You should not be seeing this since the upper directory is removed beforehand")

You should not be seeing this since the upper directory is removed beforehand


In [4]:
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
    """Function to split the data into train/test
    First checks that the image exists in the folder - this is a safety net for any images which don't exist that we missed when manually checking images 
    Creates a list of all images and randomises them
    Splits into train test depending on split size (defined below)
    Copies all the train images into train folder and vice versa
    """
    
    ignore = []
    for image in os.listdir(SOURCE_DIR):
        if type(cv2.imread(os.path.join(SOURCE_DIR, image))) is type(None):
            ignore.append(image)
    
    source_images = [image for image in os.listdir(SOURCE_DIR) if image not in ignore]
    # Randomising list
    source_images = random.sample(source_images, len(source_images))
    
    train_images = source_images[:int(SPLIT_SIZE * len(source_images))]
    val_images = source_images[int(SPLIT_SIZE * len(source_images)):]
    for image in train_images:
        shutil.copyfile(os.path.join(SOURCE_DIR, image), os.path.join(TRAINING_DIR, image))
    for image in val_images:
        shutil.copyfile(os.path.join(SOURCE_DIR, image), os.path.join(VALIDATION_DIR, image))
        

In [5]:
split_size = 0.7

#Loops over all the poses, defines the directories for source/train/test
#Splits the data into train/test
#Prints how many images are in train/test and in the original directory - sanity check!

for pose in poses_list:
    if pose == "Training" or pose == "Testing":
        pass
    else:
        SOURCE_DIR = f"{root_dir}/{pose}"
        TRAINING_DIR = f"{root_dir}/Training/{pose}"
        TESTING_DIR = f"{root_dir}/Testing/{pose}"    
        split_data(SOURCE_DIR, TRAINING_DIR, TESTING_DIR, split_size)

        print(f"There are {len(os.listdir(TRAINING_DIR))} images for {pose} in training")
        print(f"There are {len(os.listdir(TESTING_DIR))} images for {pose} in testing")

        print(f"Original {pose}'s directory has {len(os.listdir(SOURCE_DIR))} images\n")

There are 190 images for Garland_Pose in training
There are 107 images for Garland_Pose in testing
Original Garland_Pose's directory has 236 images



libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


There are 100 images for Peacock_Pose in training
There are 55 images for Peacock_Pose in testing
Original Peacock_Pose's directory has 177 images

There are 173 images for Four-Limbed_Staff in training
There are 95 images for Four-Limbed_Staff in testing
Original Four-Limbed_Staff's directory has 203 images



libpng warning: iCCP: known incorrect sRGB profile
Corrupt JPEG data: 21 extraneous bytes before marker 0xd9


There are 546 images for Sitting_pose_1_(normal) in training
There are 306 images for Sitting_pose_1_(normal) in testing
Original Sitting_pose_1_(normal)'s directory has 637 images

There are 63 images for Staff_Pose_ in training
There are 35 images for Staff_Pose_ in testing
Original Staff_Pose_'s directory has 72 images

There are 217 images for Low_Lunge_pose in training
There are 125 images for Low_Lunge_pose in testing
Original Low_Lunge_pose's directory has 295 images

There are 154 images for Standing_big_toe_hold in training
There are 88 images for Standing_big_toe_hold in testing
Original Standing_big_toe_hold's directory has 228 images



In [6]:
#These variables can be changes, excluding train_dir

train_dir = "../raw_data/Training"
img_height, img_width = 256, 256
batch_size = 32

In [7]:
#Splits into train_generator and validation_generator
#This bulk uploads the images
#Creates target (y) for us!

#Play around with the interpolation argument - bicubic, lanczos??? 

train_datagen = ImageDataGenerator(rescale=1./255,
                                    vertical_flip=True,
                                    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
                                    train_dir,
                                    target_size=(img_height, img_width),
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    subset='training',
                                    keep_aspect_ratio=True,
                                    interpolation='lanczos') # set as training data

validation_generator = train_datagen.flow_from_directory(
                                    train_dir, # same directory as training data
                                    target_size=(img_height, img_width),
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    subset='validation',
                                    keep_aspect_ratio=True,
                                    interpolation='lanczos') # set as validation data

Found 1157 images belonging to 7 classes.
Found 286 images belonging to 7 classes.


# Model

In [35]:
...
# Model needs building + transfer learning  
...
def initialize_model():
    base_model = efficientnet.EfficientNetB0(
                        include_top=False,
                        weights='imagenet',
                        input_shape=(img_height, img_width, 3),
                        classifier_activation='softmax')
    
    base_model.trainable = False
    
    model = models.Sequential([ 
        base_model,
        layers.Flatten(),
        layers.Dense(500, activation="relu"),
        layers.Dense(7, activation="softmax")
    ])
    
    
    opt = optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt,
                loss='categorical_crossentropy',
                metrics=['accuracy']) 
    
    return model

In [33]:
model = initialize_model()

In [30]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 8, 8, 1280)       4049571   
                                                                 
 flatten_2 (Flatten)         (None, 81920)             0         
                                                                 
 dense_4 (Dense)             (None, 500)               40960500  
                                                                 
 dense_5 (Dense)             (None, 83)                41583     
                                                                 
Total params: 45,051,654
Trainable params: 41,002,083
Non-trainable params: 4,049,571
_________________________________________________________________


In [39]:
es = EarlyStopping(patience=1)

In [36]:
#fit model - fit on train_generator (both X and y) and the validation data is validation_generator
history = model.fit(
                train_generator,
                steps_per_epoch = train_generator.samples // batch_size,
                validation_data = validation_generator, 
                validation_steps = validation_generator.samples // batch_size,
                epochs = 2,
                callbacks=[es])

Epoch 1/2
36/36 [==============================] - 28s 784ms/step - loss: 1.9057 - accuracy: 0.3307 - val_loss: 1.9142 - val_accuracy: 0.3828
Epoch 2/2
36/36 [==============================] - 29s 787ms/step - loss: 2.0593 - accuracy: 0.3049 - val_loss: 1.8824 - val_accuracy: 0.3711


# Bounding Boxes

In [ ]:
#This has been paused and probably won't be used

In [ ]:
im = cv2.imread(f"{file_path}/{file_list[0]}")

In [37]:
# Reading the Image
image = cv2.imread(f"{file_path}/{file_list[3]}")

# initialize the HOG descriptor
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

(humans, _) = hog.detectMultiScale(image, winStride=(3,3),
                                padding=(32, 32), 
                               scale=1.01)

# getting no. of human detected
print('Human Detected : ', len(humans))

# loop over all detected humans
for (x, y, w, h) in humans:
   pad_w, pad_h = int(0.15 * w), int(0.01 * h)
   cv2.rectangle(image, (x + pad_w, y + pad_h), (x + w - pad_w, y + h - pad_h), (0, 255, 0), 2)



Human Detected :  2


In [38]:
# display the output image
cv2.imshow("Image", image)
cv2.waitKey(25)
# cv2.destroyAllWindows()

102

In [41]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [50]:
filepath = f"{file_path}/{file_list[3]}"
photo = cv2.imread(filepath)
photo_grey = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)

In [ ]:
# Initializing the HOG person
# detector

hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
   
# Reading the Image
image = cv2.imread(filepath)
   
# Detecting all the regions in the 
# Image that has a pedestrians inside it
(regions, _) = hog.detectMultiScale(image, 
                                    winStride=(8, 8),
                                    padding=(16, 16),
                                    scale=1.15)
   
# Drawing the regions in the Image
for (x, y, w, h) in regions:
    cv2.rectangle(image, (x, y), 
                  (x + w, y + h), 
                  (0, 0, 255), 2)
    
#Showing the output Image
cv2.imshow("Image", image)
cv2.waitKey(0)
   
cv2.destroyAllWindows()